In [22]:
import os
from utils.config import GRAPH_SAVE_FOLDER, DATA_FOLDER
from utils.plotter import plot_line_comparison


In [23]:
# import pandas as pd

# data_dir = os.path.join(DATA_FOLDER)
# extra_tag = f"Starlink_channel_specs_250MHz_10.5GHz"    
# tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# # Ensure the tagged folder exists
# os.makedirs(tagged_folder, exist_ok=True)

# df = pd.read_csv('Satellite_Australia_Simulation_Log.csv')
# df = pd.read_csv('./250mhz_Data_10.5Ghz/Satellite_Australia_Simulation_Log.csv')

In [24]:
# import pandas as pd

# data_dir = os.path.join(DATA_FOLDER)
# extra_tag = f"Starlink_BGAN"    
# tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# # Ensure the tagged folder exists
# os.makedirs(tagged_folder, exist_ok=True)

# df = pd.read_csv('Satellite_Australia_Simulation_Log.csv')
# # df = pd.read_csv('./250mhz_Data_10.5Ghz/Satellite_Australia_Simulation_Log.csv')

In [25]:
import pandas as pd

data_dir = os.path.join(DATA_FOLDER)
extra_tag = f"starlink_downlink"    
tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# Ensure the tagged folder exists
os.makedirs(tagged_folder, exist_ok=True)

df = pd.read_csv('Satellite_Australia_Simulation_Log_starlink_downlink.csv')

In [26]:
# import pandas as pd

# data_dir = os.path.join(DATA_FOLDER)
# extra_tag = f"starlink_uplink"    
# tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# # Ensure the tagged folder exists
# os.makedirs(tagged_folder, exist_ok=True)

# df = pd.read_csv('Satellite_Australia_Simulation_Log_starlink_uplink.csv')

In [27]:
import pandas as pd
import re

# Step 1: Drop columns that are all zeros or all NaN
df = df.drop(columns=[col for col in df.columns if ((df[col] == 0) | (df[col].isna())).all()])

# Step 2: Get max satellite ID from column names
sat_ids = [int(match.group(1)) for col in df.columns if (match := re.match(r'LEO(\d+)_', col))]
max_sat_id = max(sat_ids) if sat_ids else -1

print("Max satellite ID found:", max_sat_id)

# Step 3: Track satellites with access
possible_sat_conn_ids = []

for i in range(max_sat_id + 1):
    sat_id = f'LEO{i}'

    # Check access columns
    sydney_access_col = f'{sat_id}_Sydney_Access'
    melbourne_access_col = f'{sat_id}_Melbourne_Access'

    # Check if access columns exist and have any True values
    # .fillna(False) ensures we treat NaN as False
    # .any() checks if there is at least one True value
    sydney_access_ok = sydney_access_col in df.columns and df[sydney_access_col].fillna(False).any()
    melbourne_access_ok = melbourne_access_col in df.columns and df[melbourne_access_col].fillna(False).any()

    if sydney_access_ok or melbourne_access_ok:
        possible_sat_conn_ids.append(sat_id)
    else:
        # Drop base satellite info if no access
        base_cols = [f'{sat_id}_Name', f'{sat_id}_Lat', f'{sat_id}_Lon', f'{sat_id}_Freq_Hz']
        existing_cols_to_drop = [col for col in base_cols if col in df.columns]
        df = df.drop(columns=existing_cols_to_drop)

print("Satellites with possible connection:", possible_sat_conn_ids)
print("length(Satellites with possible connection):", len(possible_sat_conn_ids))


Max satellite ID found: 144
Satellites with possible connection: ['LEO20', 'LEO21', 'LEO22', 'LEO23', 'LEO24', 'LEO31', 'LEO32', 'LEO33', 'LEO34', 'LEO35', 'LEO42', 'LEO43', 'LEO44', 'LEO45', 'LEO46', 'LEO54', 'LEO55', 'LEO56', 'LEO57', 'LEO65', 'LEO66', 'LEO67', 'LEO68', 'LEO69', 'LEO76', 'LEO77', 'LEO78', 'LEO79', 'LEO80', 'LEO88']
length(Satellites with possible connection): 30


In [28]:
# Collect all satellite-related columns NOT in possible_sat_conn_ids to drop
cols_to_drop = []

for col in df.columns:
    # Extract satellite id number from column name if it matches pattern
    match = re.match(r'(LEO\d+)_', col)
    if match:
        sat_id = match.group(1)
        # If satellite not in possible_sat_conn_ids, mark its columns for deletion
        if sat_id not in possible_sat_conn_ids:
            cols_to_drop.append(col)

# Drop those columns
df = df.drop(columns=cols_to_drop)

print("Remaining columns after cleanup:", df.columns.tolist())


Remaining columns after cleanup: ['Time', 'LEO20_Name', 'LEO20_Lat', 'LEO20_Lon', 'LEO20_Freq_Hz', 'LEO20_Sydney_Access', 'LEO20_Sydney_SNR_dB', 'LEO20_Sydney_RSSI_dBm', 'LEO20_Sydney_Throughput', 'LEO20_Sydney_BER_QPSK', 'LEO20_Sydney_BER_MQAM', 'LEO20_Sydney_Latency', 'LEO20_Sydney_TimeOut', 'LEO20_Melbourne_Access', 'LEO20_Melbourne_SNR_dB', 'LEO20_Melbourne_RSSI_dBm', 'LEO20_Melbourne_Throughput', 'LEO20_Melbourne_BER_QPSK', 'LEO20_Melbourne_BER_MQAM', 'LEO20_Melbourne_Latency', 'LEO20_Melbourne_TimeOut', 'LEO21_Name', 'LEO21_Lat', 'LEO21_Lon', 'LEO21_Freq_Hz', 'LEO21_Sydney_Access', 'LEO21_Sydney_SNR_dB', 'LEO21_Sydney_RSSI_dBm', 'LEO21_Sydney_Throughput', 'LEO21_Sydney_BER_QPSK', 'LEO21_Sydney_BER_MQAM', 'LEO21_Sydney_Latency', 'LEO21_Sydney_TimeOut', 'LEO21_Melbourne_Access', 'LEO21_Melbourne_SNR_dB', 'LEO21_Melbourne_RSSI_dBm', 'LEO21_Melbourne_Throughput', 'LEO21_Melbourne_BER_QPSK', 'LEO21_Melbourne_BER_MQAM', 'LEO21_Melbourne_Latency', 'LEO21_Melbourne_TimeOut', 'LEO22_Name'

In [30]:
df.to_csv('Satellite_Australia_Simulation_Log_cleaned.csv')

In [8]:
all_rssi_cols = [col for col in df.columns if re.search(r'_RSSI_dBm$', col)]
all_snr_cols = [col for col in df.columns if re.search(r'_SNR_dB$', col)]


all_thrpt_cols = [col for col in df.columns if re.search(r'_Throughput$', col)]
all_BER_QPSK_cols = [col for col in df.columns if re.search(r'_BER_QPSK$', col)]
all_BER_MQAM_cols = [col for col in df.columns if re.search(r'_BER_MQAM$', col)]
all_Latency_cols = [col for col in df.columns if re.search(r'Latency$', col)]
all_TimeOut_cols = [col for col in df.columns if re.search(r'TimeOut$', col)]

In [9]:
df[all_thrpt_cols] = df[all_thrpt_cols] / (1024 * 1024) # Convert to Mbps

In [10]:
df[all_Latency_cols] = df[all_Latency_cols] * 1000 # Convert to Mbps

In [11]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build Latency column names
    sydney_Latency_col = f'{sat_id}_Sydney_Latency'
    melbourne_Latency_col = f'{sat_id}_Melbourne_Latency'

    # Check which Latency columns exist
    sydney_exists = sydney_Latency_col in df.columns
    melbourne_exists = melbourne_Latency_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    temp_tagged_folder = os.path.join(tagged_folder, "Latency")
    # Plot 
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_Latency_col],
            labels=[sydney_Latency_col],
            xlabel='Time (m)',
            ylabel='Latency (ms)',
            title=f'Latency for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_latency_.png",
            folder=temp_tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_Latency_col],
            labels=[melbourne_Latency_col],
            xlabel='Time (m) ',
            ylabel='Latency (ms)',
            title=f'Latency for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Melbourne_latency_.png",
            folder=temp_tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [12]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build Thrpt column names
    sydney_Thrpt_col = f'{sat_id}_Sydney_Throughput'
    melbourne_Thrpt_col = f'{sat_id}_Melbourne_Throughput'

    # Check which Thrpt columns exist
    sydney_exists = sydney_Thrpt_col in df.columns
    melbourne_exists = melbourne_Thrpt_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    temp_tagged_folder = os.path.join(tagged_folder, "thrpt")
    # Plot 
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_Thrpt_col],
            labels=[sydney_Thrpt_col],
            xlabel='Time (m)',
            ylabel='Thrpt (Mbps)',
            title=f'Thrpt for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_thrpt_.png",
            folder=temp_tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_Thrpt_col],
            labels=[melbourne_Thrpt_col],
            xlabel='Time (m) ',
            ylabel='Thrpt (Mbps)',
            title=f'Thrpt for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Melbourne_thrpt_.png",
            folder=temp_tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [13]:
df.head(2)

,Time,LEO20_Name,LEO20_Lat,LEO20_Lon,LEO20_Freq_Hz,LEO20_Sydney_Access,LEO20_Sydney_SNR_dB,LEO20_Sydney_RSSI_dBm,LEO20_Sydney_Throughput,LEO20_Sydney_BER_QPSK,...,LEO88_Freq_Hz,LEO88_Sydney_Access,LEO88_Sydney_SNR_dB,LEO88_Sydney_RSSI_dBm,LEO88_Sydney_Throughput,LEO88_Sydney_BER_QPSK,LEO88_Sydney_BER_MQAM,LEO88_Sydney_Latency,LEO88_Sydney_TimeOut,LEO88_Melbourne_TimeOut
0,10-Apr-2025 12:00:00,_20,-25.639253,2.791160,11100000000,0,NaN,NaN,NaN,NaN,...,11700000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,10-Apr-2025 12:00:30,_20,-27.041493,4.078029,11300000000,0,NaN,NaN,NaN,NaN,...,11100000000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [14]:
for i, row in df.iterrows():
    # Get the highest value across all RSSI columns at the `i-th` row
    highest_rssi_at_i = df[all_rssi_cols].iloc[i].max()

    # Get the column name that has the highest RSSI value at the `i-th` index
    column_with_highest_rssi = df[all_rssi_cols].iloc[i].idxmax()

    print(f"Highest RSSI value at index {i}: {highest_rssi_at_i}")
    print(f"Column with the highest RSSI value at index {i}: {column_with_highest_rssi}")

Highest RSSI value at index 0: -115.764485236438
Column with the highest RSSI value at index 0: LEO35_Melbourne_RSSI_dBm
Highest RSSI value at index 1: -114.715293404594
Column with the highest RSSI value at index 1: LEO35_Melbourne_RSSI_dBm
Highest RSSI value at index 2: -116.812429716472
Column with the highest RSSI value at index 2: LEO35_Sydney_RSSI_dBm
Highest RSSI value at index 3: -114.96097014057
Column with the highest RSSI value at index 3: LEO68_Melbourne_RSSI_dBm
Highest RSSI value at index 4: -114.646178386542
Column with the highest RSSI value at index 4: LEO68_Melbourne_RSSI_dBm
Highest RSSI value at index 5: -114.568660235191
Column with the highest RSSI value at index 5: LEO68_Melbourne_RSSI_dBm
Highest RSSI value at index 6: -112.222252667522
Column with the highest RSSI value at index 6: LEO68_Melbourne_RSSI_dBm
Highest RSSI value at index 7: -112.342794503145
Column with the highest RSSI value at index 7: LEO68_Melbourne_RSSI_dBm
Highest RSSI value at index 8: -117.

In [15]:
stats = df[all_rssi_cols].fillna(0).describe().T
stats.to_csv(os.path.join('./', 'rssi_stats.csv'))

In [16]:
stats = df[all_rssi_cols].describe().T
stats.to_csv(os.path.join('./', 'rssi_stats.csv'))

In [17]:
stats = df[all_snr_cols].describe().T
stats.to_csv(os.path.join('./', 'snr_stats.csv'))

In [18]:

# Highest value per column
max_per_column = df[all_rssi_cols].max()

# Highest value among all columns
overall_max = df[all_rssi_cols].max().max()

# Displaying results
print("\nHighest Value per Column:\n", max_per_column)
print("\nHighest Value Among All Columns:", overall_max)


Highest Value per Column:
 LEO20_Sydney_RSSI_dBm      -148.295443
LEO20_Melbourne_RSSI_dBm   -134.922692
LEO21_Sydney_RSSI_dBm      -137.625636
LEO21_Melbourne_RSSI_dBm   -129.573836
LEO22_Sydney_RSSI_dBm      -131.267904
LEO22_Melbourne_RSSI_dBm   -127.442422
LEO23_Sydney_RSSI_dBm      -125.463062
LEO23_Melbourne_RSSI_dBm   -123.662092
LEO24_Sydney_RSSI_dBm      -129.809117
LEO24_Melbourne_RSSI_dBm   -139.395803
LEO31_Sydney_RSSI_dBm      -134.859752
LEO31_Melbourne_RSSI_dBm   -121.556846
LEO32_Sydney_RSSI_dBm      -110.554143
LEO32_Melbourne_RSSI_dBm   -108.541788
LEO33_Sydney_RSSI_dBm      -112.088481
LEO33_Melbourne_RSSI_dBm   -107.525019
LEO34_Sydney_RSSI_dBm      -115.756424
LEO34_Melbourne_RSSI_dBm   -110.302245
LEO35_Sydney_RSSI_dBm      -115.995717
LEO35_Melbourne_RSSI_dBm   -114.715293
LEO42_Melbourne_RSSI_dBm   -150.120775
LEO43_Sydney_RSSI_dBm      -128.704046
LEO43_Melbourne_RSSI_dBm   -127.825126
LEO44_Sydney_RSSI_dBm      -134.485834
LEO44_Melbourne_RSSI_dBm   -126.9481

In [19]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build SNR column names
    sydney_SNR_col = f'{sat_id}_Sydney_SNR_dB'
    melbourne_SNR_col = f'{sat_id}_Melbourne_SNR_dB'

    # Check which SNR columns exist
    sydney_exists = sydney_SNR_col in df.columns
    melbourne_exists = melbourne_SNR_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    # Plot 
    temp_tagged_folder = os.path.join(tagged_folder, "SNR")
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_SNR_col],
            labels=[sydney_SNR_col],
            xlabel='Time (m)',
            ylabel='SNR (dBm)',
            title=f'SNR for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_SNR_.png",
            folder=temp_tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_SNR_col],
            labels=[melbourne_SNR_col],
            xlabel='Time (m) ',
            ylabel='SNR (dBm)',
            title=f'SNR for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Melbourne_SNR_.png",
            folder=temp_tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [20]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build RSSI column names
    sydney_RSSI_col = f'{sat_id}_Sydney_RSSI_dBm'
    melbourne_RSSI_col = f'{sat_id}_Melbourne_RSSI_dBm'

    # Check which RSSI columns exist
    sydney_exists = sydney_RSSI_col in df.columns
    melbourne_exists = melbourne_RSSI_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    # Plot 
    temp_tagged_folder = os.path.join(tagged_folder, "SNR")
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_RSSI_col],
            labels=[sydney_RSSI_col],
            xlabel='Time (m)',
            ylabel='RSSI(dBm)',
            title=f'RSSI for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_RSSI_.png",
            folder=temp_tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_RSSI_col],
            labels=[melbourne_RSSI_col],
            xlabel='Time (m) ',
            ylabel='RSSI(dBm)',
            title=f'RSSI for Starlink Satellites',
            filename=f"{extra_tag}_{sat_id}_Melbourne_RSSI_.png",
            folder=temp_tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [21]:
# import matplotlib.pyplot as plt
# import re

# all_rssi_cols = [col for col in df.columns if re.search(r'_RSSI_dBm$', col)]
# all_snr_cols = [col for col in df.columns if re.search(r'_SNR_dB$', col)]

# # Loop over each satellite in possible_sat_conn_ids
# for sat_id in possible_sat_conn_ids:
#     # Build RSSI column names
#     sydney_RSSI_col = f'{sat_id}_Sydney_RSSI_dBm'
#     melbourne_RSSI_col = f'{sat_id}_Melbourne_RSSI_dBm'

#     # Check which RSSI columns exist
#     sydney_exists = sydney_RSSI_col in df.columns
#     melbourne_exists = melbourne_RSSI_col in df.columns

#     # Skip if neither exists
#     if not sydney_exists and not melbourne_exists:
#         continue

#     plt.figure(figsize=(10, 5))
#     plt.title(f'RSSI vs Time for {sat_id}')
#     plt.xlabel('Time')
#     plt.ylabel('RSSI (dB)')

#     if sydney_exists:
#         plt.plot(df['Time'], df[sydney_RSSI_col], label='Sydney', color='blue')
#     if melbourne_exists:
#         plt.plot(df['Time'], df[melbourne_RSSI_col], label='Melbourne', color='green')

#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()
